In [16]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

def load_documents():
    document_loader = PyPDFDirectoryLoader("data/")
    return document_loader.load()

In [17]:
documents = load_documents()

In [32]:
print(documents[0])

page_content='TheGALE
ENCYCLOPEDIA
ofMEDICINE
SECOND EDITION' metadata={'source': 'data\\four.pdf', 'page': 0}
